Generating code for a model:

In [1]:
import pytorch_composer
import pytorch_composer.datasets


# A random sequence of neural network layers. Any positive integer shoud be a valid dimension arguement:
sequence = [
    ["Conv2d", 6],
    ["Reshape",(4,32,32)],
    ["MaxPool2d", 2],
    ["Linear", 16],
    ["RNN",12,{"batch_first":True}],
    ["Relu"],
    ["MaxPool2d", 2],
    ["Linear",43],
    ["MaxPool2d", 2],
    ["Relu"],
    ["Flat"],
    ["Linear",38],
]
dataset = pytorch_composer.datasets.CIFAR10()
model = pytorch_composer.Model(sequence, dataset)
loop = pytorch_composer.TrainingLoop(model)

code = pytorch_composer.Code([dataset,model,loop])
code

import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim


# Load and normalize the dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class Net(nn.Module):


Using the generated code:

In [2]:
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim


# Load and normalize the dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv2d1 = nn.Conv2d(3, 6, 3)
        self.adaptiveavgpool2d1 = nn.AdaptiveAvgPool2d((32, 32))
        self.maxpool2d1 = nn.MaxPool2d(2, stride=2)
        self.linear1 = nn.Linear(16, 16)
        self.rnn1 = nn.RNN(16, 12, batch_first=True)
        self.adaptiveavgpool2d2 = nn.AdaptiveAvgPool2d((1, 196))
        self.linear2 = nn.Linear(7, 43)
        self.linear3 = nn.Linear(63, 38)

    def forward(self, x, h1):
        # Input dimensions : (4, 3, 32, 32)
        # Convolution layer (2d): (4, 3, 32, 32) -> (4, 6, 30, 30)
        x = self.conv2d1(x)
        x = x.view(1, 4, 6, 900)
        # Resizing with adaptive average pooling: (1, 4, 6, 900) -> (1, 4, 32, 32)
        x = self.adaptiveavgpool2d1(x)
        x = x.view(4, 32, 32)
        x = x.view(4, 1, 32, 32)
        # Pooling layer (2d max): (4, 1, 32, 32) -> (4, 1, 16, 16)
        x = self.maxpool2d1(x)
        # Linear layer: (4, 1, 16, 16) -> (4, 1, 16, 16)
        x = self.linear1(x)
        x = x.view(4, 16, 16)
        # Recurrent layer: (4, 16, 16) -> (4, 16, 12)
        x, h1 = self.rnn1(x, h1)
        x = F.relu(x)
        x = x.view(1, 4, 16, 12)
        # Resizing with adaptive average pooling: (1, 4, 16, 12) -> (1, 4, 1, 196)
        x = self.adaptiveavgpool2d2(x)
        x = x.view(4, 1, 14, 14)
        # Pooling layer (2d max): (4, 1, 14, 14) -> (4, 1, 7, 7)
        x = self.maxpool2d1(x)
        # Linear layer: (4, 1, 7, 7) -> (4, 1, 7, 43)
        x = self.linear2(x)
        # Pooling layer (2d max): (4, 1, 7, 43) -> (4, 1, 3, 21)
        x = self.maxpool2d1(x)
        x = F.relu(x)
        x = x.view(4, 63)
        # Linear layer: (4, 63) -> (4, 38)
        x = self.linear3(x)
        # Output dimensions : (4, 38)
        return x, h1

    def initHidden(self):
        h1 = torch.zeros(1, 4, 12)
        return h1

# Define a Loss function and optimizer
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


# Training
for epoch in range(2):  # loop over the dataset multiple times
    h1 = net.initHidden()

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs, h1 = net(inputs, h1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        h1 = h1.data

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
        torch.save(net.state_dict(), 'model.pt')

print('Finished Training')

Files already downloaded and verified
Files already downloaded and verified
[1,  2000] loss: 2.434
[1,  4000] loss: 2.165
[1,  6000] loss: 2.102
[1,  8000] loss: 2.066
[1, 10000] loss: 2.032
[1, 12000] loss: 2.003
[2,  2000] loss: 1.964
[2,  4000] loss: 1.948
[2,  6000] loss: 1.912
[2,  8000] loss: 1.896
[2, 10000] loss: 1.884
[2, 12000] loss: 1.875
Finished Training
